<a href="https://colab.research.google.com/github/khoda81/dethcod/blob/main/DETHCOD_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    

In [1]:
import os
import wandb


# huggingface_hub.login(token=userdata.get('huggigface_token'))

try:
    from dotenv import load_dotenv
    # Load environment variables from .env file
    load_dotenv()

except ImportError as e:
    print(f"Error importing dotenv: {e}")


# Check if running in Colab
try:
    from google.colab import userdata
    # If running in Colab, use userdata.get to retrieve the token
    wandb.login(key=userdata.get('wandb_token'))

except ImportError:
    # If not in Colab, load the token from the environment variable
    wandb_token = os.getenv('WANDB_TOKEN')
    if wandb_token:
        wandb.login(key=wandb_token)
    else:
        print("W&B token not found in environment variable. Please set WANDB_TOKEN in your environment.")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\20mah\.netrc
wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.


In [3]:
%run -m dethcod.dataset

In [ ]:
%run -i -m dethcod.vector.train --batch-size 2

In [1]:
from dethcod.vector.model import VectorCompressionModel

model = VectorCompressionModel.from_pretrained("data/models/vector-t5-dethcod/")

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")

In [2]:
import datasets

data_files = ["data/dataset/enwik8"]
dataset = datasets.load_dataset("text", data_files=data_files)
dataset = dataset["train"]

In [3]:
import random
sample = random.choice(dataset)

In [ ]:
from transformers import GenerationConfig
from transformers.modeling_outputs import BaseModelOutput

input_ids = tokenizer.encode(
    sample["text"],
    return_tensors="pt",
    padding=True,
    truncation=True,
).input_ids.to(model.device)

print("Input Sequence: ")
print(repr(tokenizer.decode(input_ids[0])))
print()

model_output = model.forward(input_ids)
print("loss:", model_output.loss.item())
print()

print("Compressing...")
compressed_form = model.compress(input_ids)

print("Decompressing...")
last_hidden_state = model.unpooling_layer.forward(compressed_form)
encoder_outputs = BaseModelOutput(last_hidden_state=last_hidden_state)
generation_config = GenerationConfig(
    do_sample=True,
    num_return_sequences=16,
    max_new_tokens=100,
)

generation_output = model.generate(
    encoder_outputs=encoder_outputs,
    generation_config=generation_config,
)

print("Generated Sequences: ")
for seq in generation_output.tolist():
    seq = [token for token in seq if token != 0]
    print(repr(tokenizer.decode(seq)))


Input Sequence: 
'* [[Combustion]], in which a substance reacts with an oxidizing element, such as [[oxygen]] gas.</s>'

loss: 1.5137027502059937

Generated Sequences: 
'* [[Combussion methylation, oxygen]], oxygenes, which offer a reaction, to</s>'
'* Interactions in [[gas]], which have fueling fuel through oxygen, and fuel to fuel, the substance of that substance in a combustions.</s>'
'* In exchange reaction reaction to elements, [[oxidation]] oxygenes, containing a substance, which gases in oxidation.</s>'
'* Besides a composting substance, oxygening compounds to aceas, airing (reaction), and a [[combustion]]</s>'
'* <unk>reactions in the compound-sicing process, like hydrogen, [[oxygen]], [[olinex]], in which anoxygens matter in which</s>'
"* The inoxygenation reaction comes into a component of Oxgenitine, such as ''oxygenoideline in air, such as fuel</s>"
'* In more gases, oxygens in oxidation like a reaction containing [[cinemit], in a substance, in substance in a form</s>'
'* i